In [1]:
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, LSTM,Conv2D, MaxPooling2D, Lambda
from keras import models
from keras import layers
from keras import optimizers
from keras import regularizers
from keras import initializers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize

import numpy as np
import os
import random
import cv2



Using TensorFlow backend.


In [2]:
def getVideoFileList(path):
    file_list = []
    dir_list = os.listdir(path)
    
    for _dir in dir_list:
        files=os.listdir(path+_dir)

        for file in files:
            if(file.split("-")[0] =='01'):
                file_list.append(_dir+"/"+file)
                
    random.shuffle(file_list)
    return file_list

def getAudioFileListfromVideoFile(path, file_list):
    audio_file_list = []
    
    for file in file_list:
        audio_file = file[:9] + '03' + file[11:30] + "wav"
        audio_file_list.append(audio_file)
    
    return audio_file_list

def getEmotionListfromFileName(file_list):
    emotion_list = []
    
    for file in file_list:
        emotion_list.append(int(file.split("-")[2]))
    
    return emotion_list

#video_path = "../dataset/RAVDESS/Video/"
#audio_path = "../dataset/RAVDESS/Audio/"

#video_file_list = getVideoFileList(video_path)
#audio_file_list = getAudioFileListfromVideoFile(audio_path, video_file_list)
#emotion_list = getEmotionListfromFileName(video_file_list)
#print(emotion_list)
#print(len(video_file_list))
#print(len(emotion_list))
#print(audio_file_list)


In [3]:
#video_file_list 불러오기
f = open("../video_file_list_20200316.txt", 'r')

video_file_list = []

while True:
    line = f.readline()
    if not line: break
        
    video_file_list.append(line[:-1])
    
f.close()

video_path = "../dataset/RAVDESS/Video/"
audio_path = "../dataset/RAVDESS/Audio/"
audio_file_list = getAudioFileListfromVideoFile(audio_path, video_file_list)
emotion_list = getEmotionListfromFileName(video_file_list)

### 이미지 시퀸스 불러오기
+ 순차적으로 이미지 불러와 동기를 맞춤

In [4]:
path = "../dataset/RAVDESS/Images/"

video_frames = []

for i in range(len(video_file_list)):
    _dir, filename = video_file_list[i].split("/")
    
    new_dir_name = path+_dir+"/"+filename.replace(".","_")
    #print(new_dir_name)
    dir_list = os.listdir(new_dir_name)
    
    frames = []
    for j in range(len(dir_list)):
        file = str(j)+".jpg"
        gray = cv2.imread(new_dir_name + "/" + file, cv2.IMREAD_GRAYSCALE) / 255.0
        
        frames.append(gray)
    
    video_frames.append(frames) 
#print(np.shape(video_frames[1]))

### Landmark 시퀸스 불러오기
+ 순차적으로 이미지를 불러와 동기를 맞춤

In [5]:
path = "../dataset/RAVDESS/Landmarks/"

face_landmarks=[]
for i in range(len(video_file_list)):
    _dir, filename = video_file_list[i].split("/")
    
    new_dir_name = path+_dir+"/"+filename.replace(".","_")
    
    dir_list = os.listdir(new_dir_name)
    
    frames=[]
    for j in range(len(dir_list)):
        file = str(j)+".txt"
        
        f= open(new_dir_name+"/"+file, "r")
        landmark = []
        
        while True:
            line=f.readline()
            if not line: 
                break
                
            if line != "":
                landmark.append(int(line))
                
        frames.append(landmark)
        
    face_landmarks.append(frames)
#print(face_landmarks[0][0]) #

#pre-processing
for i in range(len(face_landmarks)):
    for j in range(len(face_landmarks[i])):
        x_s = np.array(face_landmarks[i][j][0::2])
        y_s = np.array(face_landmarks[i][j][1::2])
        
        x_o = x_s[16]
        y_o = y_s[16]
        
        x_std = np.std(x_s)
        y_std = np.std(y_s)
        
        for k in range(0, len(face_landmarks[i][j]), 2):
            face_landmarks[i][j][k] = (face_landmarks[i][j][k] - x_o) /x_std
            
        for k in range(1, len(face_landmarks[i][j]), 2):
            face_landmarks[i][j][k] = (face_landmarks[i][j][k] - y_o) /y_std

### Speech data 불러오기

In [6]:
sr = 48000
window_size = 1600

path = "../dataset/RAVDESS/Audio_Features/"

audio_features = []
for i in range(len(audio_file_list)):
    _dir, filename = audio_file_list[i].split("/")
    
    new_dir_name = path+_dir+"/"+filename.replace(".","_")
    
    dir_list = os.listdir(new_dir_name)
    
    frames=[]
    for j in range(len(dir_list)):
        file = str(j)+".txt"
        
        f= open(new_dir_name+"/"+file, "r")
        features = []
        
        while True:
            line=f.readline()
            if not line: 
                break
                
            if line != "":
                features.append(float(line))
                
        frames.append(features)
        
    audio_features.append(frames)

In [7]:
#parameter setting
class_size =8 #neutral, calm, happy, sad, angry, fearful, disgust, surprised
time_step = 10
sliding_value = int(time_step/2)
image_x = []
landmark_x = []
audio_x = []
y = []

#### 이미지 sequence 데이터 

In [8]:
# 이미지 sequence 데이터
for i in range(len(video_frames)):
    for j in range(0, len(video_frames[i]) - time_step + 1, sliding_value): #j는 시간
        temp = video_frames[i][j:j+time_step]
        image_x.append(temp)
        y.append(emotion_list[i])
#print(np.shape(image_x), np.shape(y)) #((데이터길이), 10(time_steps), 64, 64(64*64이미지)) (3273, )

X_train = np.array(image_x)
X_train = X_train.reshape(len(X_train),time_step,64,64,1)
Y_train = np.array(y)
Y_train = label_binarize(Y_train, classes = [1, 2, 3, 4, 5, 6, 7, 8])
#print(Y_train[0])

#### Landmark 데이터

In [9]:
#Landmark 데이터
for i in range(len(face_landmarks)):
    for j in range(0, len(face_landmarks[i])- time_step + 1, sliding_value):
        temp = face_landmarks[i][j:j+time_step]
        landmark_x.append(temp)
        y.append(emotion_list[i])

Landmark_X_train = []

#for i in range(len(landmark_x)):
#    x = []
#    for j in range(len(landmark_x[i])):
#        x.extend(landmark_x[i][j])
        
#    Landmark_X_train.append(x)
    
Landmark_X_train = np.array(landmark_x)
print(np.shape(Landmark_X_train), np.shape(Y_train))

(13838, 10, 98) (13838, 8)


#### Speech 데이터

In [10]:
for i in range(len(audio_features)):
    for j in range(0, len(audio_features[i])- time_step + 1, sliding_value):
        temp = audio_features[i][j:j+time_step]
        audio_x.append(temp)
        y.append(emotion_list[i])

Audio_X_train = np.array(audio_x)
print(np.shape(Audio_X_train))

(13838, 10, 43)


### CNN+LSTM model
+ video_frames -> CNN -> featuremap
+ output_featuremap + face_landmarks -> LSTM

#### input shape

In [11]:
print (np.shape(X_train), np.shape(Landmark_X_train), np.shape(Audio_X_train), np.shape(Y_train))

(13838, 10, 64, 64, 1) (13838, 10, 98) (13838, 10, 43) (13838, 8)


In [12]:
X_test = np.array(X_train[:1300])
X_train = np.array(X_train[1300:])
Landmark_X_test = np.array(Landmark_X_train[:1300])
Landmark_X_train = np.array(Landmark_X_train[1300:])
Audio_X_test = np.array(Audio_X_train[:1300])
Audio_X_train = np.array(Audio_X_train[1300:])
Y_test = np.array(Y_train[:1300])
Y_train = np.array(Y_train[1300:])

In [13]:
print (np.shape(X_train),np.shape(X_test))

(12538, 10, 64, 64, 1) (1300, 10, 64, 64, 1)


In [14]:
print (np.shape(Landmark_X_train), np.shape(Landmark_X_test))

(12538, 10, 98) (1300, 10, 98)


In [15]:
print (np.shape(Audio_X_train),np.shape(Audio_X_test))

(12538, 10, 43) (1300, 10, 43)


In [16]:
print (np.shape(Y_train), np.shape(Y_test))

(12538, 8) (1300, 8)


#### z normalization(speech)

In [13]:
feature_size = 43

x_for_scaling = []
avg = []
std = []

#training feature를 feature별로
for i in range(feature_size):
    temp = []
    x_for_scaling.append(temp)
    
for i in range(len(Audio_X_train)):
    for j in range(len(Audio_X_train[i])):
        for k in range(len(Audio_X_train[i][j])):
            x_for_scaling[k].append(Audio_X_train[i][j][k])
            
#각 feature의 평균과 표준편차를 구함
for i in range(len(x_for_scaling)):
    avg.append(np.mean(x_for_scaling[i]))
    std.append(np.std(x_for_scaling[i]))
    
#training 벡터 normalization
for i in range(len(Audio_X_train)):
    for j in range(len(Audio_X_train[i])):
        for k in range(len(Audio_X_train[i][j])):
            Audio_X_train[i][j][k] = (Audio_X_train[i][j][k] - avg[k])/std[k]
            
#test 벡터 normalization
for i in range(len(Audio_X_test)):
    for j in range(len(Audio_X_test[i])):
        for k in range(len(Audio_X_test[i][j])):
            Audio_X_test[i][j][k] = (Audio_X_test[i][j][k] - avg[k])/std[k]

#### model

In [17]:
input_x = layers.Input(shape = (time_step, 64, 64,1))
input_x_landmark = layers.Input(shape = (time_step, 98))
input_x_audio = layers.Input(shape = (time_step, 43))

stack = []    

for i in range(time_step):
    in_layer = Lambda(lambda x : x[:,i,:,:,:]) (input_x)
    conv_layer1 = layers.Conv2D(8,(3,3), activation = "relu")(in_layer)
    pooling1 = layers.MaxPooling2D((2,2))(conv_layer1)
    conv_layer2 = layers.Conv2D(16,(3,3), activation = "relu")(pooling1)
    pooling2 = layers.MaxPooling2D((2,2))(conv_layer2) #shape = (데이터길이, 14,14,16)

    flat = layers.Flatten()(pooling2)
    stack.append(flat)


lstm_in1 = Lambda(lambda x : tf.stack(x,axis=1))(stack) #shape = (?,10,? )
lstm_in2 = Lambda(lambda x : tf.concat((x,input_x_landmark),2))(lstm_in1)
lstm_in3 = Lambda(lambda x : tf.concat((x,input_x_audio),2))(lstm_in2)
lstm1 = layers.LSTM(64, input_shape = (time_step, 14*14*16+98+43))(lstm_in3) #lstm_ 입력 shape = datasize, timestep, 14*14*16+98+43
#lstm2 = layers.LSTM(64)(lstm1) return_sequences = True,
dense1 = layers.Dense(32, activation='relu')(lstm1)
dropout1 = layers.Dropout(0.5)(dense1)
dense2 = layers.Dense(class_size)(dropout1)
lstm_out = layers.Softmax()(dense2)

model = Model([input_x, input_x_landmark, input_x_audio], lstm_out)
model.compile(optimizer = 'adam', loss = "categorical_crossentropy", metrics=['accuracy'])

###
#model.fit(X_train, Y_train, epochs = 50, batch_size = 100)
model.fit([X_train, Landmark_X_train, Audio_X_train], Y_train, epochs = 50, batch_size = 100)

avg_acc=0.0
acc = model.evaluate([X_test, Landmark_X_test, Audio_X_test], Y_test)
#avg_acc+=acc[1]
print(acc[1])



Epoch 1/50
12538/12538 [==============================] - 8s 619us/step - loss: 2.0056 - accuracy: 0.1924
Epoch 2/50
12538/12538 [==============================] - 6s 444us/step - loss: 1.6905 - accuracy: 0.3806
Epoch 3/50
12538/12538 [==============================] - 6s 446us/step - loss: 1.2818 - accuracy: 0.5483
Epoch 4/50
12538/12538 [==============================] - 6s 444us/step - loss: 1.0124 - accuracy: 0.6491
Epoch 5/50
12538/12538 [==============================] - 6s 446us/step - loss: 0.8065 - accuracy: 0.7335
Epoch 6/50
12538/12538 [==============================] - 6s 447us/step - loss: 0.6440 - accuracy: 0.7932
Epoch 7/50
12538/12538 [==============================] - 6s 447us/step - loss: 0.4967 - accuracy: 0.8460
Epoch 8/50
12538/12538 [==============================] - 6s 447us/step - loss: 0.4424 - accuracy: 0.8643
Epoch 9/50
12538/12538 [==============================] - 6s 447us/step - loss: 0.3245 - accuracy: 0.9022
Epoch 10/50
12538/12538 [====================

+ video_file_list_20200316 => 0.76
+ video_file_list_20200318 => 0.77
+ video_file_list_20200319 => 0.81

In [16]:
#a=tf.concat([stack,input_x_landmark],axis=1)
pooling2

<tf.Tensor 'max_pooling2d_20/MaxPool:0' shape=(?, 14, 14, 16) dtype=float32>

In [18]:
stack

[<tf.Tensor 'flatten_1/Reshape:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'flatten_2/Reshape:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'flatten_3/Reshape:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'flatten_4/Reshape:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'flatten_5/Reshape:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'flatten_6/Reshape:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'flatten_7/Reshape:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'flatten_8/Reshape:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'flatten_9/Reshape:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'flatten_10/Reshape:0' shape=(?, ?) dtype=float32>]

In [19]:
lstm_in1

<tf.Tensor 'lambda_11/stack:0' shape=(?, 10, ?) dtype=float32>

In [20]:
lstm_in2

<tf.Tensor 'lambda_12/concat:0' shape=(?, 10, ?) dtype=float32>

In [22]:
a =tf.concat((lstm_in1,input_x_landmark),2)
print(a)

Tensor("concat_1:0", shape=(?, 10, ?), dtype=float32)
